Primeiramente, vamos importar as bibliotecas e os dados necessários.

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
random.seed(42)

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/ozlerhakan/ab-test/master/ab_data.csv')
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


### Encontre o número de linhas no conjunto de dados.




In [3]:
df.shape

(294478, 5)

### Encontre o número de usuários únicos no conjunto de dados.


In [5]:
df.user_id.nunique()

290584

### Encontre a proporção de usuários convertidos.

In [6]:
df.query('converted == 1')['converted'].count() / df.shape[0]

0.11965919355605512

### Cheque o número de vezes que `new_page` e `treatment` não se alinham.

In [7]:
df.query('group == "treatment" and landing_page != "new_page"').count()[0]

1965

In [8]:
df.query('group != "treatment" and landing_page == "new_page"').count()[0]

1928

In [9]:
1965 + 1928

3893

### Para as linhas em que o tratamento não está alinhado com `new_page` ou o controle não está alinhado com `old_page`, não podemos ter certeza se essa linha realmente recebeu a página nova ou antiga. Por isso, vamos manipular um pouco o dataset.

In [10]:
df2 = df.iloc[df.query('group == "treatment" and landing_page == "new_page"').index.values]

In [12]:
df3 = df.iloc[df.query('group == "control" and landing_page == "old_page"').index.values]

In [13]:
df2 = df2.append(df3)

In [14]:
df2[((df2['group'] == 'treatment') == (df2['landing_page'] == 'new_page')) == False].shape[0]

0

### O usuário `773192` está duplicado. Cheque isso e remova uma das entradas

In [17]:
df2.query('user_id == 773192')

,user_id,timestamp,group,landing_page,converted
1899,773192,2017-01-09 05:37:58.781806,treatment,new_page,0
2893,773192,2017-01-14 02:55:59.590927,treatment,new_page,0


In [18]:
df2.drop(1899, inplace=True)

### Qual é a probabilidade de um indivíduo converter, independentemente da página que recebe?

In [19]:
df2.converted.mean()

0.11959708724499628

### Dado que um indivíduo estava no grupo de controle, qual é a probabilidade de conversão?

In [20]:
control_convert = df2.query('group == "control"').converted.mean()
control_convert

0.1203863045004612

### Dado que um indivíduo estava no grupo de tratamento, qual é a probabilidade de conversão?

In [21]:
treatment_convert = df2.query('group == "treatment"').converted.mean()
treatment_convert

0.11880806551510564

### Qual é a probabilidade de que um indivíduo tenha recebido a nova página?

In [22]:
df2.query('landing_page == "new_page"').count()[0] / df2.shape[0]

0.5000619442226688

### Qual é a probabilidade de que um indivíduo tenha recebido a página antiga?

In [23]:
1 - 0.5000619442226688

0.4999380557773312

Observe que, devido ao carimbo de data/hora associado a cada evento, você pode tecnicamente executar um teste de hipótese continuamente à medida que cada observação é observada.

No entanto, a pergunta difícil é: você para assim que uma página é considerada significativamente melhor que outra ou precisa acontecer de forma consistente por um determinado período de tempo? Quanto tempo você corre para renderizar uma decisão de que nenhuma página é melhor que a outra?

Essas perguntas são as partes difíceis associadas aos testes A/B em geral.

Por enquanto, considere que você precisa tomar a decisão apenas com base em todos os dados fornecidos. Se você quiser supor que a página antiga é melhor, a menos que a nova página seja definitivamente melhor com uma taxa de erro Tipo I de 5% (ou seja, `0,05`), quais devem ser suas hipóteses nula e alternativa? Você pode declarar sua hipótese em termos de palavras ou em termos de e , que são as taxas convertidas para as páginas antigas e novas.

![](https://miro.medium.com/max/592/1*ZfVY8b4r8UjUprgOe7ITVA.jpeg)

Assuma sob a hipótese nula, `p_old` e `p_new` ambos têm taxas de sucesso "verdadeiras" iguais à taxa de sucesso convertida, independentemente da página - ou seja, e são iguais. Além disso, suponha que sejam iguais à taxa convertida em ab_data.csv, independentemente da página.

Use um tamanho de amostra para cada página igual aos de ab_data.csv.

Execute a distribuição de amostragem para a diferença na conversão entre as duas páginas em mais de 10.000 iterações de cálculo de uma estimativa a partir do nulo.

Use as células abaixo para fornecer as partes necessárias desta simulação. Se isso não fizer todo o sentido agora, não se preocupe - você trabalhará com os problemas abaixo para concluir isso.

In [24]:
df2.landing_page.value_counts()

new_page    145310
old_page    145274
Name: landing_page, dtype: int64

### Qual é a taxa de conversão `p_new` para sob a nula?

In [25]:
p_new = df2.converted.mean()
p_new

0.11959708724499628

### Qual é a taxa de conversão `p_old` para sob a nula?

In [26]:
p_old = p_new
p_old

0.11959708724499628

### Agora vamos calcular `n_new` e `n_old`

In [27]:
n_new = df2.query('landing_page == "new_page"').count()[0] 
n_new

145310

In [28]:
n_old = df2.query('landing_page == "old_page"').count()[0]
n_old

145274

### Agora vamos utilizar uma biblioteca para calcular o p-valor

In [29]:
import statsmodels.api as sm

# o número de conversões para cada página
convert_old = df2.query('landing_page == "old_page"').converted.sum()  # old page == control group
convert_new = df2.query('landing_page == "new_page"').converted.sum()  # new page == treatment group
# o número de indivíduos que receberam cada página
n_old = df2.query('landing_page == "old_page"')['user_id'].nunique()
n_new = df2.query('landing_page == "new_page"')['user_id'].nunique()

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


### Agora use `stats.proportions_ztest` para calcular sua estatística de teste e valor-p. 

In [30]:
z_score, p_value = sm.stats.proportions_ztest([convert_new, convert_old], [n_new, n_old], alternative = 'larger')

In [31]:
from scipy.stats import norm

norm.cdf(z_score) # Nos diz o quão significativo é o nosso z-score

0.09494168724097551

In [32]:
norm.ppf(1-(0.05/2)) # Nos diz qual é o nosso valor crítico com 95% de confiança

1.959963984540054

### O que o z-score e o p-value que você calculou na pergunta anterior significam para as taxas de conversão das páginas antigas e novas?

In [34]:
#O valor p calculado usando proporções_ztest é 0,9. Como sabemos que 90% dos valores da nossa distribuição nula caem para a direita ou estão acima da nossa média real nas partes j e k. 
#Nesse caso, falharíamos em rejeitar a hipótese nula como 0,9 > 0,05.

#Em outras palavras, como o z-score de -1,3109241984234394 não excede o valor crítico de 0,9050583127590245, 
#não rejeitaríamos a hipótese nula de que a página antiga tem uma taxa de conversão melhor ou igual que a nova página.